In [4]:
# plot usnig plotly
import plotly.graph_objects as go
import pandas as pd

hist_data = pd.read_csv('EURUSD_15min_data.csv')
fig = go.Figure(data=[go.Candlestick(x=hist_data['dt'],
                open=hist_data['open'],
                high=hist_data['high'],
                low=hist_data['low'],
                close=hist_data['close'])])

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed